# Pesquisando por iniciativas/projetos com Dados Abertos Governamentais no Github

In [1]:
import requests
import pandas as pd
import time

TODO:

**Mapeamento das principais bibliotecas**

- [API da Câmara dos deputados](https://dadosabertos.camara.leg.br/swagger/api.html)

In [2]:
search_strings_libraries = []

Devido a estrutura de pesquisa no github as palavras chaves utilizadas para o 
news não foram tão eficiente, visto que preciso adicionar palavras de busca ou 
de forma conter uma das palavras ou conter todas as palavras na sequencia especificada.

Sendo assim:
- '"projeto" and "dados governo"',
- '"projeto" and "dados governamentais"',
- '"projeto" and "monitora" and "dados" and "governo"'

Não foram buscas efetivas, porém as palavras chaves inicias sim.

In [3]:
search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

'consumir' e 'publicação' não retornaram resultados e por isso foram retirados a fim de adicionar palavras que retornem resultados melhores
- 'consumir dados abertos do governo'
- 'consumir dados abertos governamentais'
- 'publicação de dados abertos do governo',
- 'publicação de dados governamentais',

In [4]:
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?q='
credentials=('lorenaps','ba6db863961f1221ef38f7294976f0dc8dc36731')

Verificando o limite de requisições

In [5]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1573173555},
 'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1573173555},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1573173555},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1573173555},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1573170015}}}

In [6]:
colunas=[
        'id',
        'full_name',
        'description',
        'owner_type', 
        'owner_api_url',
        'owner_url',
        'url',
        'api_url',
        'fork',
        'created_at',
        'updated_at',
        'size',
        'stargazers_count',
        'language',
        'has_issues',
        'has_wiki',
        'forks_count',
        'forks',
        'open_issues',
        'watchers',
        'timestamp_extract'
    ]


In [7]:
def add_resultado(item):
    
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('full_name', None),
                        item.get('description', None),      
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None),
                        str(time.time()).split('.')[0]]], columns=colunas)

    return df    

In [8]:
def extrair_dados(data, resultados):
    
    for item in data.get('items', None):
        
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)

    return resultados

Verificando limitação de extração de dados da API

In [9]:
mais_de_mil = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(mais_de_mil, auth=credentials)
t.json()

{'documentation_url': 'https://developer.github.com/v3/search/',
 'message': 'Only the first 1000 search results are available'}

In [10]:
resultados = pd.DataFrame(columns=['id',
                                    'full_name',
                                    'description',
                                    'owner_type', 
                                    'owner_api_url',
                                    'owner_url',
                                    'url',
                                    'api_url',
                                    'fork',
                                    'created_at',
                                    'updated_at',
                                    'size',
                                    'stargazers_count',
                                    'language',
                                    'has_issues',
                                    'has_wiki',
                                    'forks_count',
                                    'forks',
                                    'open_issues',
                                    'watchers',
                                    'timestamp_extract',
                                    'commits',
                                    'contributors',]) 

In [11]:
def percorrendo_paginas(url, resultados):
    
    print('\nPrimeira requisição')
    
    results = requests.get(url, auth=credentials)    
    data = dict(results.json())
    total = data.get('total_count', None)
        
    print(">>> Foram encontrados {0} resultados. Extraindo...".format(total))

    resultados = extrair_dados(data, resultados)
    
    iteracoes = total // 30 
    
    for iteracao in range(0, iteracoes):        
        header = dict(results.links)
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
            
            print("\nNext url: {0}".format(next_url))
            
            results = requests.get(next_url, auth=credentials)
            data = dict(results.json())
            resultados = extrair_dados(data, resultados)
        
    return resultados

In [12]:
for string in search_strings:

    print("\nExtraindo repositórios para a string: '{0}'".format(string))

    url = url_base + string + sort
    %%time
    resultados = percorrendo_paginas(url, resultados)


Extraindo repositórios para a string: 'dados abertos'
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs

Primeira requisição
>>> Foram encontrados 370 resultados. Extraindo...

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=2

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=3

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=4

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=5

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=6

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=7

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=8

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=9

Next url: https:/

In [13]:
resultados.describe()

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
count,570,570,547,570,570,570,570,570,570,570,...,420,570,570,570,570,570,570,570,0.0,0.0
unique,395,395,372,2,340,340,395,395,1,395,...,22,2,2,16,16,14,25,31,0.0,0.0
top,195294598,thenets/ckanext-dadosabertos,Plugin/Tema do Portal de Dados Abertos do Gove...,User,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/prodest/ckanext-data_es_theme,https://api.github.com/repos/thenets/ckanext-d...,False,2016-08-17T11:33:20Z,...,Python,True,True,0,0,0,0,1573169982,NaN,NaN
freq,6,6,6,456,16,16,6,6,570,6,...,91,555,554,424,424,490,377,30,NaN,NaN


## Extraindo Commits e Contributors

In [28]:
def extract_commits(url_repo):
    
    commits_url = url_repo + '/commits'
        
    results = requests.get(commits_url, auth=credentials)
    commits = len(results.json())

    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')        
        results = requests.get(next_url, auth=credentials)
        commits = commits + len(results.json())    
        header = dict(results.links)


    return commits

In [29]:
def extract_contributors(url_repo):
    
    contributors_url = url_repo + '/contributors'
    
    results = requests.get(contributors_url, auth=credentials)
    contributors = len(results.json())

    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        results = requests.get(next_url, auth=credentials)
        contributors = contributors + len(results.json())
        header = dict(results.links)
    
    return contributors

In [ ]:
urls = resultados['api_url']

for url in urls:
        
    print('\n>>> ', url)
    
    repo = requests.get(url, auth=credentials)
    repo = dict(repo.json())
    
    if repo.get('size', 0) is not 0:
        
        commits = extract_commits(url)
        contributors = extract_contributors(url)

        print("Tem {0} Commits - {1} Contributors".format(commits,contributors))
    
        resultados.loc[resultados["api_url"] == url, 'commits'] = commits
        resultados.loc[resultados["api_url"] == url, 'contributors'] = contributors
        
    else:
        print('> Repositório tem size igual a 0')


>>>  https://api.github.com/repos/CamaraDosDeputados/dados-abertos
Tem 28 Commits - 4 Contributors

>>>  https://api.github.com/repos/dadosgovbr/catalogos-dados-brasil
Tem 42 Commits - 4 Contributors

>>>  https://api.github.com/repos/prefeiturasp/dados-educacao
Tem 18 Commits - 2 Contributors

>>>  https://api.github.com/repos/dadosgovbr/aplicativos-dados-brasil
Tem 40 Commits - 5 Contributors

>>>  https://api.github.com/repos/dadosgovbr/kit
Tem 361 Commits - 6 Contributors

>>>  https://api.github.com/repos/mapaslivres/localidades
Tem 69 Commits - 5 Contributors

>>>  https://api.github.com/repos/odufrn/odufrn-downloader
Tem 237 Commits - 6 Contributors

>>>  https://api.github.com/repos/brasilopen/brasilopen
Tem 10 Commits - 2 Contributors

>>>  https://api.github.com/repos/vitorbaptista/dados-abertos-camara.gov.br
Tem 31 Commits - 1 Contributors

>>>  https://api.github.com/repos/jonny-data/conheca-seu-vereador
Tem 70 Commits - 6 Contributors

>>>  https://api.github.com/repos/da

In [ ]:
resultados.head(5)

In [ ]:
file_path = '../data/repositories_' + str(time.time()).split('.')[0] + '.csv'
file_path

In [ ]:
resultados.to_csv(file_path, index=False)

## Extraindo contribuidores dos repositórios

In [ ]:
colunas_contributors = [ 'repo_id',
                          'repo_name',
                          'repo_url',
                          'repo_api_url',
                          'contributor_id',
                          'contributor_login',
                          'contributor_type'
                          'contributor_url',
                          'contributor_api_url',
                          'timestamp_extract']

In [ ]:
def get_contributors(url):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    results = results.json()
    
    for result in results:
        contributor = {}
        
        contributor = {
            'contributor_id': result.get('id', None),
            'contributor_login': result.get('login', None),
            'contributor_type': result.get('type', None),
            'contributor_url': result.get('html_url', None),
            'contributor_api_url': result.get('url', None),
        }

        list_contributors.append(contributor)
        
    return list_contributors

In [ ]:
def add_contributor(repo, contributor):
            
    df = pd.DataFrame([[
                    repo.get('repo_id'),
                    repo.get('repo_name', None),
                    repo.get('repo_url', None),      
                    repo.get('repo_api_url', None),
                    contributor.get('contributor_id', None),
                    contributor.get('contributor_login', None),
                    contributor.get('contributor_type', None),
                    contributor.get('contributor_url', None),
                    contributor.get('contributor_api_url', None),
                    str(time.time()).split('.')[0]]], columns=colunas_contributors)

    return df  

In [ ]:
def save_contributors(contributors_df, repo, contributors):
    
    for contributor in contributors:
        
        contributors_df = pd.concat([contributors_df, 
                                     add_contributor(repo, contributor)], 
                                    ignore_index=True, 
                                    sort=False)
    return contributors_df

In [ ]:
def search_contributors(df):
    
    contributors_df = pd.DataFrame(columns = colunas_contributors)
        
    for repo in repos:

        repo = {
            'repo_id': df['id'],
            'repo_name': df['full_name'],
            'repo_url': df['url'],
            'repo_api_url': df['api_url']
        }
        
        url_contributors = repo.get('repo_api_url') + '/contributors'        
        contributors = get_contributors(url_contributors)
        
        contributors_df = save_contributors(contributors_df, repo, contributors)
        
        return contributors_df

In [ ]:
result_contributors = search_contributors(resultados)
result_contributors.describe()

In [ ]:
file_path = '../data/contributors_' + str(time.time()).split('.')[0] + '.csv'
file_path

In [ ]:
result_contributors.to_csv(file_path, index=False)